<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon.

(2) Collect the top 10000 User Reviews of a film recently in 2023 or 2022 (you can choose any film) from IMDB.

(3) Collect all the reviews of the top 1000 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from [Semantic Scholar](https://www.semanticscholar.org).

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using a hashtag (you can use any hashtag) from Twitter. 


In [1]:
!pip install fold_to_ascii
!pip install stanza
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import matplotlib.pyplot as plt
from collections import Counter
from fold_to_ascii import fold
import nltk
from nltk.stem import PorterStemmer
nltk.download("punkt")
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("omw-1.4")
import nltk
nltk.download('averaged_perceptron_tagger')
import pandas as pd
import tweepy
from nltk import pos_tag
from nltk import RegexpParser
import spacy
from spacy import displacy
import stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=7dde3047ad782922eeded826717c4e8abf45f12410497b04f555e09e2e946827
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [16]:
# Write your code here

consumer_key ="2Em7SxlX9jPMfL4x97r3zMO0x"
consumer_secret = "sVbJzekKuiAgq83Y7gCwNVbSowqQokGVzWexKHl2cXIPceWtSd"
access_key ="1439767876962029572-uUMt8oWRyzj9ilE5zk4uYbL93sCMPT"
access_secret = "oydIGymn9bS767FVEMawE9GyGAnMmBJfaY2XXKmHnmliF"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)
df = pd.DataFrame(columns=['username', 'text'])
print("Enter Twitter HashTag to search for")
words = input()
tweet_data = tweepy.Cursor(api.search,words, lang="en",tweet_mode='extended').items(1000)
tweet_list = list()
for i in tweet_data:
  tweet_list.append(i)
for tweet in tweet_list:
  try:
    username = tweet.user.screen_name
    text = tweet.retweeted_status.full_text
  except AttributeError:
    text = tweet.full_text
    username = tweet.user.screen_name
  df.loc[len(df)] = [username,text]
df


Enter Twitter HashTag to search for
#sunday


,username,text
0,VSTMMJJ,"Chilean ceramics artist Livia Marin, from her ..."
1,Jeffers46889505,Smart kid 🤣🤣🤣🤣. Happy Sunday guys.\n\nLadi Ade...
2,_puddykat,Morning all 🥰\n\nSinful Sunday 😉\n\nLet’s make...
3,VonnyNuijten,"Chilean ceramics artist Livia Marin, from her ..."
4,sakurai_satomi,"Chilean ceramics artist Livia Marin, from her ..."
...,...,...
995,goanewshub,#Gavoredo|| A wild bison was sighted in a cash...
996,whatsn2day,"Superquiz and Target Time, Sunday, March 5 #MA..."
997,KanyaWaTumaini,Lamentations 3: 22-23. It is of the LORD's Mer...
998,dennisckurian,Once upon a time .@ArvindKejriwal .\n\n ...


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [18]:
# Write your code here

def finding_ex_c(row): #function for finding special characters
  l = list()
  a=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','\t',' ','\n']
  for i in df[row]:
    if i == 0:
      continue
    for j in i:
      j = fold(str(j))
      k=j.lower()
      for m in k:
        if(m not in a):
          l.append(m)
  return set(l)



In [19]:
def text_clean(col, mp): #function for text cleaning
  stop_words = stopwords.words('english')
  stop_words += ['why', 'where', 'what', 'when', 'who']
  title = list()
  x = ['0','2','1','3','4','5','6','7','8','9']
  for lis in df[col]:
    if lis == 0:
      title.append(0)
      continue
    y = list()
    word_lis = lis.split(" ")
    for words in word_lis :
      words = fold(words)
      if words == '':
        continue
      elif words.isdigit():
        continue
      else:
        w = ''
        for c in words:
          if c in x:
            w+= ' '
          elif c in mp:
            w += ' '
          else:
            w += c
        w = w.strip(" ")
        w = w.lower()
        if  len(w) > 0 and w not in stop_words:
          if w not in y:
            y.append(w.strip(" "))
    title.append(y)
  return title

In [20]:
sepcial_characters = finding_ex_c('text')
sepcial_characters

{'\r',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 '@',
 '_',
 '|',
 '~'}

In [21]:
clean_text = text_clean('text',sepcial_characters )

In [22]:
df['text'] = clean_text
df

,username,text
0,VSTMMJJ,"[chilean, ceramics, artist, livia, marin, noma..."
1,Jeffers46889505,"[smart, kid, happy, sunday, guys \n\nladi, ade..."
2,_puddykat,"[morning, \n\nsinful, sunday, \n\nlet s, make,..."
3,VonnyNuijten,"[chilean, ceramics, artist, livia, marin, noma..."
4,sakurai_satomi,"[chilean, ceramics, artist, livia, marin, noma..."
...,...,...
995,goanewshub,"[gavoredo, wild, bison, sighted, cashew, plant..."
996,whatsn2day,"[superquiz, target, time, sunday, march, time\..."
997,KanyaWaTumaini,"[lamentations, lord s, mercies, consumed, comp..."
998,dennisckurian,"[upon, time, arvindkejriwal, \n\n, now\n\n, st..."


In [23]:
#stemming
stemer = PorterStemmer() 
l = list()
for word_list in df['text']:
  p = list()
  for word in word_list:
   p.append(stemer.stem(word))
  l.append(p)
df['text'] = l
df

,username,text
0,VSTMMJJ,"[chilean, ceram, artist, livia, marin, nomad, ..."
1,Jeffers46889505,"[smart, kid, happi, sunday, guys \n\nladi, ade..."
2,_puddykat,"[morn, \n\nsin, sunday, \n\nlet , make, \n\n s..."
3,VonnyNuijten,"[chilean, ceram, artist, livia, marin, nomad, ..."
4,sakurai_satomi,"[chilean, ceram, artist, livia, marin, nomad, ..."
...,...,...
995,goanewshub,"[gavoredo, wild, bison, sight, cashew, plantat..."
996,whatsn2day,"[superquiz, target, time, sunday, march, time\..."
997,KanyaWaTumaini,"[lament, lord , merci, consum, compass, fail, ..."
998,dennisckurian,"[upon, time, arvindkejriw, \n\n, now\n\n, stay..."


In [24]:
# Lemmatization
lemma = WordNetLemmatizer() 
l = list()
for word_list in df['text']:
  p = list()
  for word in word_list:
    p.append(lemma.lemmatize(word, pos='v'))
  l.append(p)
df['text'] = l
df

,username,text
0,VSTMMJJ,"[chilean, ceram, artist, livia, marin, nomad, ..."
1,Jeffers46889505,"[smart, kid, happi, sunday, guys \n\nladi, ade..."
2,_puddykat,"[morn, \n\nsin, sunday, \n\nlet , make, \n\n s..."
3,VonnyNuijten,"[chilean, ceram, artist, livia, marin, nomad, ..."
4,sakurai_satomi,"[chilean, ceram, artist, livia, marin, nomad, ..."
...,...,...
995,goanewshub,"[gavoredo, wild, bison, sight, cashew, plantat..."
996,whatsn2day,"[superquiz, target, time, sunday, march, time\..."
997,KanyaWaTumaini,"[lament, lord , merci, consum, compass, fail, ..."
998,dennisckurian,"[upon, time, arvindkejriw, \n\n, now\n\n, stay..."


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [25]:
# POS
c = Counter()
for text in df['text']:
  tokens_tag = pos_tag(text)
  print(tokens_tag)
  for i in tokens_tag:
    c[i[1]] += 1




[('chilean', 'JJ'), ('ceram', 'NN'), ('artist', 'NN'), ('livia', 'NN'), ('marin', 'NN'), ('nomad', 'JJ'), ('patternsseri', 'JJ'), ('womensart', 'NN'), ('sunday', 'NN'), ('https   t co  i lzk lmx', 'NN')]
[('smart', 'JJ'), ('kid', 'NN'), ('happi', 'NN'), ('sunday', 'JJ'), ('guys \n\nladi', 'NN'), ('adebutu', 'VBP'), ('us', 'PRP'), ('uk', 'JJ'), ('okek', 'JJ'), ('cynthia', 'NN'), ('joshua', 'NN'), ('selman', 'JJ'), ('idoma', 'NN'), ('david', 'NN'), ('hundeyin', 'NN'), ('na', 'TO'), ('pos\n\n sundayservic', 'JJ'), ('sundaymass', 'NN'), ('sundaypost', 'NN'), ('smac', 'NN'), ('chrisrockl', 'NN'), ('shugat', 'NN'), ('festivaldacancao', 'NN'), ('lalistarplusl', 'NN'), ('livmun', 'NN'), ('https   t co lobkkugdnc', 'NN')]
[('morn', 'JJ'), ('\n\nsin', 'NNP'), ('sunday', 'NN'), ('\n\nlet ', 'NNP'), ('make', 'VBP'), ('\n\n sunday', 'JJ'), ('https   t co ncesglx oq', 'NN')]
[('chilean', 'JJ'), ('ceram', 'NN'), ('artist', 'NN'), ('livia', 'NN'), ('marin', 'NN'), ('nomad', 'JJ'), ('patternsseri', 'JJ

In [26]:
#count for each POS element
print(c)

Counter({'NN': 9504, 'JJ': 2765, 'VBP': 565, 'NNP': 518, 'VBD': 334, 'VB': 296, 'RB': 223, 'IN': 211, 'NNS': 146, 'VBZ': 123, 'JJS': 101, 'CD': 97, 'FW': 68, 'VBN': 64, 'MD': 53, 'PRP': 38, 'RBS': 38, 'WP': 31, 'RBR': 25, 'CC': 22, 'JJR': 19, 'VBG': 13, 'DT': 10, 'TO': 5, 'WRB': 4, 'RP': 3, 'WP$': 2, 'PRP$': 1, 'UH': 1})


In [27]:
# dependency parsing
nlp=spacy.load('en_core_web_sm')

for text in df['text']: 
  sen = ""
  for j in text:
    sen = sen+j+" "
  dc = nlp(sen)
  for i in dc:
    print("Token\t\t|Relation\t\t|Head\t\t|Children")
    print("-------------------------------------------")
    print(i.text,"\t\t|",i.dep_,"\t\t|",i.head.text,"\t\t|",[j for i in i.children])
  print("\n")

Streaming output truncated to the last 5000 lines.
-------------------------------------------

 		| dep 		| wizard 		| []
Token		|Relation		|Head		|Children
-------------------------------------------
grizzl 		| dobj 		| wizard 		| []


Token		|Relation		|Head		|Children
-------------------------------------------
it 		| nsubj 		| sundayros 		| ['https   t co vbox rg gq']
Token		|Relation		|Head		|Children
-------------------------------------------
  		| dep 		| it 		| []
Token		|Relation		|Head		|Children
-------------------------------------------
sundayros 		| ROOT 		| sundayros 		| ['https   t co vbox rg gq', 'https   t co vbox rg gq', 'https   t co vbox rg gq', 'https   t co vbox rg gq']
Token		|Relation		|Head		|Children
-------------------------------------------
petal 		| amod 		| possibl 		| []
Token		|Relation		|Head		|Children
-------------------------------------------
pistacchio 		| compound 		| possibl 		| []
Token		|Relation		|Head		|Children
--------------------------

In [28]:
# Consistency parsing
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')
for text in df['text']:
  rp = ''
  for j in df['text'][0]:
    rp = rp+' '+j
  doc = nlp(rp)
  for sentence in doc.sentences:
      print(sentence.constituency)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: constituency
INFO:stanza:Done loading processors!


(ROOT (S (NP (NP (FW chilean) (FW ceram) (FW artist) (FW livia) (FW marin) (FW nomad)) (FW patternsseri) (FW womensart) (FW sunday)) (VP (FW https) (GW t) (GW co) (GW i) (GW lzk) (ADD lmx))))
(ROOT (S (NP (NP (FW chilean) (FW ceram) (FW artist) (FW livia) (FW marin) (FW nomad)) (FW patternsseri) (FW womensart) (FW sunday)) (VP (FW https) (GW t) (GW co) (GW i) (GW lzk) (ADD lmx))))
(ROOT (S (NP (NP (FW chilean) (FW ceram) (FW artist) (FW livia) (FW marin) (FW nomad)) (FW patternsseri) (FW womensart) (FW sunday)) (VP (FW https) (GW t) (GW co) (GW i) (GW lzk) (ADD lmx))))
(ROOT (S (NP (NP (FW chilean) (FW ceram) (FW artist) (FW livia) (FW marin) (FW nomad)) (FW patternsseri) (FW womensart) (FW sunday)) (VP (FW https) (GW t) (GW co) (GW i) (GW lzk) (ADD lmx))))
(ROOT (S (NP (NP (FW chilean) (FW ceram) (FW artist) (FW livia) (FW marin) (FW nomad)) (FW patternsseri) (FW womensart) (FW sunday)) (VP (FW https) (GW t) (GW co) (GW i) (GW lzk) (ADD lmx))))
(ROOT (S (NP (NP (FW chilean) (FW ceram)

In [29]:
# named entity recognition
NER = spacy.load("en_core_web_sm")
for text in df['text']:
  rp = ''
  for j in text:
    rp = rp+' '+j
  doc = NER(rp)
  for sentence in doc.ents:
      print("Token\t\t|Entity")
      print("-----------------")
      print(sentence.text,"\t|",sentence.label_)

Streaming output truncated to the last 5000 lines.
Token		|Entity
-----------------
karonbeach phuket 	| LOC
Token		|Entity
-----------------
thailand 	| GPE
Token		|Entity
-----------------
https 	| PERSON
Token		|Entity
-----------------
gavoredo 	| ORG
Token		|Entity
-----------------
tendulkar https 	| PERSON
Token		|Entity
-----------------
sunday 	| DATE
Token		|Entity
-----------------
sunday 	| DATE
Token		|Entity
-----------------
rescuedog https 	| PERSON
Token		|Entity
-----------------
sunday 	| DATE
Token		|Entity
-----------------
luv ranjan 	| PERSON
Token		|Entity
-----------------
tomorrow 	| DATE
Token		|Entity
-----------------
sunday 	| DATE
Token		|Entity
-----------------
day week 	| DATE
Token		|Entity
-----------------
sunday 	| DATE
Token		|Entity
-----------------
https 	| PERSON
Token		|Entity
-----------------
sunday 	| DATE
Token		|Entity
-----------------
passeng 	| PERSON
Token		|Entity
-----------------
sunday 	| DATE
Token		|Entity
-----------------
sun

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

Constituency Parsing Tree:
Using a constituency parsing tree, a sentence's syntactic structure can be represented in terms of its individual phrases. The tree is constructed by recursively breaking down a sentence into smaller components, or constituents, which are then grouped together to form larger constituents. Until the entire sentence is covered, this process is repeated. The sentence as a whole is represented by the tree's trunk, and the individual syllables are represented by the leaves. Each internal node of the tree is a representation of a constituent phrase, which is made up of one or more smaller constituents.

Dependency Parsing Tree:
The syntactic structure of a phrase can also be represented using a dependency parsing tree, although this method shows the relationships between words in terms of their dependencies rather than classifying them according to their components. Each word in a sentence is represented as a node in a dependency tree, and the connections between words are shown as directed edges connecting the nodes. A grammatical relationship, such as a subject, object, or modifier, is represented by each edge.
